# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store filepath in a variable

Import_CityDatafile = "OutputData/CityData.csv"
# Read our Data file with the pandas library

City_data_df = pd.read_csv(Import_CityDatafile)
City_data_df.head() 


,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date
0,Nichinan,31.60,131.37,23.89,83,75,0.50,JP,1595099943
1,Punta Arenas,-53.15,-70.92,2.00,80,40,8.20,CL,1595099824
2,Lagoa,39.05,-27.98,20.67,77,0,5.24,PT,1595099944
3,La Primavera,5.49,-70.41,28.92,72,26,3.59,CO,1595099944
4,Lázaro Cárdenas,17.96,-102.20,31.41,54,0,4.12,MX,1595099944


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = City_data_df[["latitude","Longitude"]]


Humidity=City_data_df["Humidity"]

In [4]:
#plot heatmap
fig= gmaps.figure(center= (0,0), zoom_level=3)

#create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=Humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=1)
#Add layer
fig.add_layer(heat_layer)

#Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:

# Plost as a Hybrid map
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=Humidity,
                                 dissipating=False,
                                 max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#get the ideal temperature using below criteria
ideal_weather = City_data_df.loc[(City_data_df["Maxtemp"].between(20,25,inclusive=True))&
                                 (City_data_df["Windspeed"]<10)&
                                 (City_data_df["Cloudiness"]==0)
                                ]
ideal_weather.head(15)

,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date
2,Lagoa,39.05,-27.98,20.67,77,0,5.24,PT,1595099944
81,Fomboni,-12.28,43.74,24.07,71,0,7.19,KM,1595099949
102,Lüderitz,-26.65,15.16,22.00,17,0,1.50,NaN,1595099950
113,San Quintín,30.48,-115.95,24.52,62,0,6.95,MX,1595099951
114,Margate,51.38,1.39,21.11,69,0,1.79,GB,1595099951
118,Walvis Bay,-22.96,14.51,22.22,14,0,3.10,NaN,1595099951
157,Arraial do Cabo,-22.97,-42.03,23.00,69,0,8.20,BR,1595099954
220,Poum,-20.23,164.02,21.36,71,0,4.96,NC,1595099961
242,Marsaxlokk,35.84,14.54,24.44,78,0,4.10,MT,1595099963
248,Vao,-22.67,167.48,20.74,75,0,6.03,NC,1595099963


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#Store into variable named hotel_df.

hotel_df=pd.DataFrame(ideal_weather)
hotel_df

,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date
2,Lagoa,39.05,-27.98,20.67,77,0,5.24,PT,1595099944
81,Fomboni,-12.28,43.74,24.07,71,0,7.19,KM,1595099949
102,Lüderitz,-26.65,15.16,22.00,17,0,1.50,NaN,1595099950
113,San Quintín,30.48,-115.95,24.52,62,0,6.95,MX,1595099951
114,Margate,51.38,1.39,21.11,69,0,1.79,GB,1595099951
118,Walvis Bay,-22.96,14.51,22.22,14,0,3.10,NaN,1595099951
157,Arraial do Cabo,-22.97,-42.03,23.00,69,0,8.20,BR,1595099954
220,Poum,-20.23,164.02,21.36,71,0,4.96,NC,1595099961
242,Marsaxlokk,35.84,14.54,24.44,78,0,4.10,MT,1595099963
248,Vao,-22.67,167.48,20.74,75,0,6.03,NC,1595099963


In [18]:
# Add columns for "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_data=hotel_df.head(10)
hotel_data

,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
2,Lagoa,39.05,-27.98,20.67,77,0,5.24,PT,1595099944,
81,Fomboni,-12.28,43.74,24.07,71,0,7.19,KM,1595099949,
102,Lüderitz,-26.65,15.16,22.00,17,0,1.50,NaN,1595099950,
113,San Quintín,30.48,-115.95,24.52,62,0,6.95,MX,1595099951,
114,Margate,51.38,1.39,21.11,69,0,1.79,GB,1595099951,
118,Walvis Bay,-22.96,14.51,22.22,14,0,3.10,NaN,1595099951,
157,Arraial do Cabo,-22.97,-42.03,23.00,69,0,8.20,BR,1595099954,
220,Poum,-20.23,164.02,21.36,71,0,4.96,NC,1595099961,
242,Marsaxlokk,35.84,14.54,24.44,78,0,4.10,MT,1595099963,
248,Vao,-22.67,167.48,20.74,75,0,6.03,NC,1595099963,


In [19]:
from pprint import pprint

In [20]:
# params dictionary to update each iteration
params = {
    
    "radius": 5000,
    #"keyword": "hotels",
    "types":"lodging",
    "key": g_key
}
# Use the lat/lng we recovered to identify hotels
for index, row in hotel_data.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["Longitude"]
 # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"    
     # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # make request and print url
    response = requests.get(base_url, params=params)
   
     # convert to json
    hotel_response= response.json()
    try:
             hotel_data.loc[index, "Hotel Name"] =  hotel_response["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    
       
hotel_data


C:\Users\Viswajani\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,latitude,Longitude,Maxtemp,Humidity,Cloudiness,Windspeed,Country,Date,Hotel Name
2,Lagoa,39.05,-27.98,20.67,77,0,5.24,PT,1595099944,Casa Das Faias
81,Fomboni,-12.28,43.74,24.07,71,0,7.19,KM,1595099949,Auberge les abou
102,Lüderitz,-26.65,15.16,22.00,17,0,1.50,NaN,1595099950,LÜDERITZ NEST HOTEL
113,San Quintín,30.48,-115.95,24.52,62,0,6.95,MX,1595099951,Old Mill Hotel
114,Margate,51.38,1.39,21.11,69,0,1.79,GB,1595099951,Premier Inn Margate hotel
118,Walvis Bay,-22.96,14.51,22.22,14,0,3.10,NaN,1595099951,Oyster Box Guesthouse
157,Arraial do Cabo,-22.97,-42.03,23.00,69,0,8.20,BR,1595099954,Pousada Porto Praia
220,Poum,-20.23,164.02,21.36,71,0,4.96,NC,1595099961,Gîte kajeon
242,Marsaxlokk,35.84,14.54,24.44,78,0,4.10,MT,1595099963,Water's Edge
248,Vao,-22.67,167.48,20.74,75,0,6.03,NC,1595099963,Hotel Kou-Bugny


In [21]:
# Save Data to csv
hotel_data.to_csv("Hotels_info.csv")


In [ ]:
#Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]
hotel_info = [info_box_template.format(**row) for index, row in hotel_data.iterrows()]
locations = hotel_data[["latitude", "Longitude"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Casa Das Faias</dd>\n<dt>City</dt><dd>Lagoa</dd>\n<dt>Country</dt><dd>PT</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Auberge les abou</dd>\n<dt>City</dt><dd>Fomboni</dd>\n<dt>Country</dt><dd>KM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>LÜDERITZ NEST HOTEL</dd>\n<dt>City</dt><dd>Lüderitz</dd>\n<dt>Country</dt><dd>nan</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Old Mill Hotel</dd>\n<dt>City</dt><dd>San Quintín</dd>\n<dt>Country</dt><dd>MX</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Premier Inn Margate hotel</dd>\n<dt>City</dt><dd>Margate</dd>\n<dt>Country</dt><dd>GB</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Oyster Box Guesthouse</dd>\n<dt>City</dt><dd>Walvis Bay</dd>\n<dt>Country</dt><dd>nan</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Pousada Porto Praia</dd>\n<dt>City</dt><dd>Arraial do Cabo</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Gîte kajeon</dd>\n<dt>City</dt><dd>Poum</dd>\n<dt>Country</dt><dd>NC</dd>\n</dl>\n',
 "\n<dl>\n<dt>Na

In [24]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))